In [1]:
#DL Model
import os, fnmatch
import sys
import librosa
import keras
import numpy as np
from keras.models import load_model

#ML Model
import pandas as pd 

from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier



# import cleaning
from sklearn.feature_selection import SelectKBest, f_classif
import pickle
import datetime

Using TensorFlow backend.
C:\Users\Manisha\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Manisha\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Manisha\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Manisha\Anaconda3\lib\site-packages\tensorflow\python\framework\d

# Pre-processing

## Helping Functions

In [3]:
def extract_features(audio_path, offset):
	y, sr = librosa.load(audio_path, offset=offset, duration=3)
	S = librosa.feature.melspectrogram(
	y, sr=sr, n_fft=2048, hop_length=512, n_mels=128)
	mfccs = librosa.feature.mfcc(S=librosa.power_to_db(S), n_mfcc=40)
	# mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
	return mfccs

def most_freq_dl(list):
    # print("Ok1")
    return np.bincount(list).argmax()
            
MAX_SOUND_CLIP_DURATION=12 #sec  
def audio_norm(data):
    max_data = np.max(data)
    min_data = np.min(data)
    data = (data-min_data)/(max_data-min_data+0.0001)
    return data-0.5

# get audio data without padding highest qualify audio
def load_file_data_without_change(folder,file_names, duration=3, sr=16000):
    input_length=sr*duration
    # function to load files and extract features
    # file_names = glob.glob(os.path.join(folder, '*.wav'))
    data = []
    for file_name in file_names:
        try:
            sound_file=folder+file_name
            print ("load file ",sound_file)
            # use kaiser_fast technique for faster extraction
            X, sr = librosa.load( sound_file,res_type='kaiser_fast') 
            dur = librosa.get_duration(y=X, sr=sr)
            # extract normalized mfcc feature from data
            mfccs_ml = np.mean(librosa.feature.mfcc(y=X, sr=sr, n_mfcc=40).T,axis=0) 
        except Exception as e:
            print("Error encountered while parsing file: ", file_name)
        feature = np.array(mfccs_ml).reshape([-1,1])
        data.append(feature)
    return data


# get audio data with a fix padding may also chop off some file
def load_file_data (folder,file_names, duration=12, sr=16000):
    input_length=sr*duration
    # function to load files and extract features
    # file_names = glob.glob(os.path.join(folder, '*.wav'))
    data = []
    for file_name in file_names:
        try:
            sound_file=folder+file_name
            print ("load file ",sound_file)
            # use kaiser_fast technique for faster extraction
            X, sr = librosa.load( sound_file, sr=sr, duration=duration,res_type='kaiser_fast') 
            dur = librosa.get_duration(y=X, sr=sr)
            # pad audio file same duration
            if (round(dur) < duration):
                print ("fixing audio lenght :", file_name)
                y = librosa.util.fix_length(X, input_length)                
            #normalized raw audio 
            # y = audio_norm(y)            
            # extract normalized mfcc feature from data
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sr, n_mfcc=40).T,axis=0)             
        except Exception as e:
            print("Error encountered while parsing file: ", file_name)        
        feature = np.array(mfccs).reshape([-1,1])
        data.append(feature)
    return data

## file to test

In [91]:
# file to test
ml_upload_folder_path = 'C:/Users/Manisha/BE_Project/Physionet_Dataset/training/training-c/'
classify_file = 'C:/Users/Manisha/BE_Project/Physionet_Dataset/training/training-c/c0022.wav'

## DL 

In [92]:
x_test_dl = []
x_test_dl.append(extract_features(classify_file, 0.5))
x_test_dl = np.asarray(x_test_dl)
# print(x_test_dl.shape[0])
# print(x_test_dl.shape[1])
# print(x_test_dl.shape[2])
# print(x_test_dl.shape[3])
x_test_dl = x_test_dl.reshape(x_test_dl.shape[0], x_test_dl.shape[1], x_test_dl.shape[2], 1)
model = keras.models.load_model('heartbeat_classifier_categorical_crossentropy.h5')
pred_class = model.predict_classes(x_test_dl, verbose=1)
CNNprob = model.predict_proba(x_test_dl, verbose=1)
# CNNprob = CNNprob[:,1:]
# print(CNNprob)
CNNpred = np.argmax(CNNprob, axis=1)
CNNpred = CNNpred + 1
# print(CNNpred)
print(type(pred_class))
scores =[]
scores = [accuracy_score(CNNpred,pred_class)]

# print(accuracy_score(CNNpred,[0]))

1/1 [==============================] - 0s 11ms/step
<class 'numpy.ndarray'>


In [78]:
x_test_dl[0][39][129][0]

2.4710030680240553

In [93]:
test_sounds = load_file_data(folder=ml_upload_folder_path,file_names=['c0022.wav'], duration=MAX_SOUND_CLIP_DURATION)
test_labels = [-1 for items in test_sounds]
testing_data_rf = np.squeeze(test_sounds)
# print(testing_data_rf)
testing_data_rf = testing_data_rf.reshape(1, -1)
loaded_model = pickle.load(open('ml_classifier_random_forest.pkl', 'rb'))
prediction_rf = loaded_model.predict(testing_data_rf)
KNNProb = loaded_model.predict_proba(testing_data_rf)
print(type(prediction_rf))

load file  C:/Users/Manisha/BE_Project/Physionet_Dataset/training/training-c/c0022.wav
<class 'numpy.ndarray'>


In [94]:
# def test(x_test_dl, testing_data_rf, dl_prediction_labels):
#     # dl model
#     pred_total = list()
#     scores = []
#     DLprob = []
#     MLprob = []
    
#     DL1model = keras.models.load_model('heartbeat_classifier_binary_crossentropy.h5')
#     DL2model = keras.models.load_model('heartbeat_classifier_categorical_crossentropy.h5')
#         model = keras.models.load_model('CHF_Ensemble_Learners/model'+str(i)+'.h5')
#         pred = model.predict_classes(x_test_dl)
#         DLprob.append(model.predict_proba(x_test_dl))
#         scores += [accuracy_score(pred,dl_prediction_labels)]
#         pred_total.append(pred)
     
    
#     loaded_model = pickle.load(open('model_rf', 'rb'))
#     prediction_rf = loaded_model.predict(testing_data_rf)
# #     probability_rf = loaded_model.predict_proba(testing_data_rf)
#     scores += [accuracy_score(prediction_rf,dl_prediction_labels)] 
#     MLprob.append(loaded_model.predict_proba(testing_data_rf))
    
#     final_pred = []
#     # CNNpred = pred_total[i]
#     # MLPpred = pred_total[i+1]
    
# #     for i in range(len(prediction_rf)):
# #         print("pred_total", pred_total)
# #         print("pred_total[i]", pred_total[i])
# #         print("pred_total[i + 1]", pred_total[i + 1])
# #         print("prediction_rf[i]", prediction_rf[i])
# #         if pred_total[i]==prediction_rf[i] and prediction_rf[i]==pred_total[i+1]:
# #             print("OK1")
# #             final_pred += [pred_total[i]]
# #         elif pred_total[i]==prediction_rf[i] or prediction_rf[i]==pred_total[i + 1] or pred_total[i]==pred_total[i + 1]:
# #             print("OK2")
# #             if pred_total[i]==prediction_rf[i] or pred_total[i]==pred_total[i + 1]:
# #                 final_pred += [pred_total[i]]
# #             elif prediction_rf[i]==pred_total[i] or prediction_rf[i]==pred_total[i + 1]:
# #                 final_pred += [prediction_rf[i]]
# #             elif pred_total[i + 1]==pred_total[i] or pred_total[i + 1]==prediction_rf[i]:
# #                 final_pred += [pred_total[i + 1]]
# #             else:
# #                 print('bug')
# #         else:
# #             print("OKK")
# # #             prob = DLprob[i] + DLprob[i + 1] + MLprob[i]
# # #             pred = np.argmax(prob)
# # #             pred = pred+1
# # #             final_pred += [pred]

#     for i in range(len(prediction_rf)):
#         if pred_total[i]==prediction_rf[i]:
#             print("yes")
#         else:
#             print("No")
        


#     final_pred = np.array(final_pred)
#     print('final_pred', final_pred)
#     print('labels: ',dl_prediction_labels)
#     score = accuracy_score(final_pred,dl_prediction_labels)
#     cm = confusion_matrix(final_pred,dl_prediction_labels)
#     return score,cm,final_pred,scores

In [95]:
# my_list = [2,5,6,7,1,9,8]
# print(my_list)
# my_list.sort()
# print(my_list[-2])

In [96]:
# dl_prediction_labels = pred_class
# acc_score, cm, final_predict, scores = test(x_test_dl, testing_data_rf, dl_prediction_labels)

# print(acc_score)
# print(final_predict)

In [97]:
# voting_clf = VotingClassifier(estimators=[('RF', loaded_model), ('CNN', model)], voting='hard')
# voting_clf.fit(X_train, y_train)
# preds = voting_clf.predict(testing_data_rf)
# acc = accuracy_score(dl_prediction_labels, preds)


In [98]:
def test_el(x_test_dl, testing_data_rf, dl_prediction_labels):
    
#     KNNmodel = MLmodel
#     MLPmodel = DL1model
#     CNNmodel = DL2model
    
    #Load saved models and PCA transformations
    MLmodel = pickle.load(open('ml_classifier_random_forest.pkl', 'rb'))
    DL1model = keras.models.load_model('heartbeat_classifier_binary_crossentropy.h5')
    DL2model = keras.models.load_model('heartbeat_classifier_categorical_crossentropy.h5')
    
    #Make predictions with test data
#     scores = []
   
    DL1pred = DL1model.predict_classes(x_test_dl)
    DL1prob = DL1model.predict_proba(x_test_dl)
#     scores += [accuracy_score(DL1pred,dl_prediction_labels)]
    DL2prob = DL2model.predict(x_test_dl)
    DL2prob = DL2prob[:,1:]
    DL2pred = np.argmax(DL2prob, axis=1)
    DL2pred = DL2pred + 1
#     scores += [accuracy_score(DL2pred,dl_prediction_labels)]
    
    MLpred = MLmodel.predict(testing_data_rf)
    MLprob = MLmodel.predict_proba(testing_data_rf)
#     scores += [accuracy_score(MLpred,dl_prediction_labels)]
    
    #Ensemble learning/voting system
    final_pred = []
    
    for i in range(len(MLpred)):
        print("DL2pred: ", DL2pred[i], "DL1pred: ", DL1pred[i], "MLpred: ", MLpred[i])
        if MLpred[i]==2:
            MLpred[i] = 0
        if DL2pred[i]==MLpred[i] and MLpred[i]==DL1pred[i]:
            final_pred += [DL2pred[i]]
        elif DL2pred[i]==MLpred[i] or MLpred[i]==DL1pred[i] or DL2pred[i]==DL1pred[i]:
            if DL2pred[i]==MLpred[i] or DL2pred[i]==DL1pred[i]:
                final_pred += [DL2pred[i]]
            elif MLpred[i]==DL2pred[i] or MLpred[i]==DL1pred[i]:
                final_pred += [MLpred[i]]
            elif DL1pred[i]==DL2pred[i] or DL1pred[i]==MLpred[i]:
                final_pred += [DL1pred[i]]
            else:
                print('bug')
        else:
            print("DL2prob[i]: ", DL2prob[i], "DL1prob[i]:", DL1prob[i], "MLprob[i]: ", MLprob[i])
            DL1prob_max = DL1prob.max()
            MLprob_max = MLprob.max()
            prob = DL2prob[i] + DL1prob_max + MLprob_max
            pred = np.argmax(prob)
            pred = pred+1
            final_pred += [pred]
    
    #Outputs
    final_pred = np.array(final_pred)
#     score = accuracy_score(final_pred,dl_prediction_labels)
#     cm = confusion_matrix(final_pred,dl_prediction_labels)
    return final_pred

In [99]:
dl_prediction_labels = pred_class

final_pred = test_el(x_test_dl, testing_data_rf, dl_prediction_labels)

print(final_pred)

DL2pred:  1 DL1pred:  1 MLpred:  2
[1]
